## Example 1

This example builds on the feature extraction from the workshop notebook, and builds upon it with some alternative feature extraction methods. Note it is _not_ required for you to fully understand what is going on with each of these methods. These can all be found in the "extract_feature" function. Quick explanation of each
- resample: stretch or compress signal such that the whole signal fits into the chosen input_length
- Min-Max Normalize: takes the biggest value and scales the signal so that all the values are between [-1, 1]. This can be helpful for Machine Learning algorithms, as they can get unstable if the values it tries to approximate have to big a difference
- log normalize: This is a technique which give more importantce to thw lower values and less importance to high values. The job here is also to make it easier for the Machine Learning algorithm to find a good model.
- welch: this is a method computing a PSD (Power Spectral Density). This is an estimate of what frequencies are present in the signal. Note that for this method, input_length _must_ be shorter than the shortest signal for it to not to crash (i.e. less than 7000 here)

### Exercise:
Using which features give you the highest validation score?

In [ ]:
from helpers import train_to_id5, load_dataset, plot_validation_history
from scipy.signal import welch, resample
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Change this to set how many steps long you want your time-series to be
input_length = 5000


# A function to extract the values we need as input and output for the model training
# Note: You can make changes here to look at different features
def extract_features(signals, train_types):
    model_input = []
    model_target = []
    
    # Iterate over all signals and corresponding train types
    for signal, train_type in zip(signals, train_types):
                
        # Assemble the signal one data point
        # ------- You can uncomment any single line below or combine some -------- #
        # signal = welch(signal, nperseg=input_length * 2)[1][1:]  # Compute an estimate of the Power Spectrum Density
        # signal = resample(signal, input_length)  # Stretch / Compress signal to fit
        # signal = signal / np.max(np.abs(signal))  # Min-Max Normalize
        # signal = np.sign(signal) * np.log(signal + 1.0)  # Reduce the range of the signal
        input_vector = np.reshape(signal, (-1, 1))  # special case if you have only 1 time series
    
        # Convert train type to number
        target = train_to_id5(train_type)
        
        # Add to dataset to be fed to a machine learning algorithm
        model_input.append(input_vector)
        model_target.append(target)
    
    # Convert to a more digestable format and return the data, also makes also signals equally long
    model_input = sequence.pad_sequences(model_input, input_length)
    model_target = np.array(model_target)
    return model_input, model_target


# Load the data
training_x, training_y = load_dataset(dataset='training')
validate_x, validate_y = load_dataset(dataset='validate')

# Transform the data / extract features
training_x, training_y = extract_features(training_x, training_y)
validate_x, validate_y = extract_features(validate_x, validate_y)

# Build a Convolutional Neural Network
# ------- You can change the number of filters and kernel_size here --------- #
model = Sequential()
model.add(Conv1D(filters=4, kernel_size=5, padding='valid', input_shape=training_x.shape[1:]))
model.add(MaxPool1D(2))
model.add(Conv1D(filters=4, kernel_size=5, padding='valid'))
model.add(MaxPool1D(2))
model.add(Flatten())
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit a model to the data. Note less epochs are needed here
# ------- You can change the number of epochs and batch_size here ----------- #
logger = model.fit(training_x, training_y, epochs=25, batch_size=16, validation_data=[validate_x, validate_y])
plot_validation_history(logger)

## Example 2

It is also possible to simply extract a couple of numbers from the signal, like the maximum value, the length of the signal and a number of other features. Then this could be fed into a simpler and tinier type of neural network.


In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from helpers import train_to_id5
from helpers import load_dataset


# A function to extract the values we need as input and output for the model training
# Note: You can make changes here to look at different features
def extract_features(signals, train_types):
    model_input = []
    model_target = []
    
    # Iterate over all signals and corresponding train types
    for signal, train_type in zip(signals, train_types):
        
        # Extract signal features with suggestion for alternative features
        signal_rms = np.sqrt(np.mean(np.square(signal)))  # Root Mean Square of the signal
        signal_mean = np.mean(signal)  # The mean value
        signal_abs_mean = np.mean(np.abs(signal))  # The mean value when all negative values are turned positive
        signal_median = np.median(signal)  # The median value
        signal_abs_median = np.median(np.abs(signal))  # The median value when all negative values are turned positive
        percentile_25 = np.percentile(signal, 25)  # 25 percentile value
        percentile_75 = np.percentile(signal, 75)  # 75 percentile value
        length = len(signal)  # number of timesteps in the signal
        
        # Assemble these values into a single data point / array
        # You can combine any or all of the above in any way you want
        input_vector = [signal_rms, signal_mean, signal_abs_mean, length]
    
        # Convert train type to number
        target = train_to_id5(train_type)
        
        # Add to dataset to be fed to a machine learning algorithm
        model_input.append(input_vector)
        model_target.append(target)
    
    # Convert to a more digestable format and return the data
    model_input = np.array(model_input)
    model_target = np.array(model_target)
    return model_input, model_target


# Load the data
training_x, training_y = load_dataset(dataset='training')
validate_x, validate_y = load_dataset(dataset='validate')

# Transform the data / extract features
training_x, training_y = extract_features(training_x, training_y)
validate_x, validate_y = extract_features(validate_x, validate_y)

# Build a simple Neural Network
model = Sequential()
model.add(Dense(units=5, input_dim=training_x.shape[1]))
model.add(Dense(units=10))
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Apply the data and the train types and have the algorithm fit a model from x to y
logger = model.fit(training_x, training_y, epochs=250, batch_size=32, validation_data=[validate_x, validate_y])

# You can add a filter after getting the model predictions to augment what it does and then validate the results
correct = 0
predicted_y = model.predict(validate_x)
for i in range(len(predicted_y)):
    # The below if statement checks if the i-th signal had a length shorter than 50000 timesteps, assuming no changes
    # were made above. keep index number in mind for easy access and additional checks if you find good filters
    # ----- You can change the value here to improve the results ----- #
    if validate_x[i][3] < 50000:
        if np.argmax(predicted_y[i]) == np.argmax(validate_y[i]):  # Compares model output with model target
            correct += 1
    else:
        if np.sum(validate_y[i]) == 0:  # Verifies that the model target is also an unknown train type
            correct += 1

# Validation accuracy after filtering
accuracy = float(correct) / float(len(predicted_y))
plot_validation_history(logger, accuracy)


## Example 3

Another thing we can try is to change the model target. Instead of treating "unknown" trains as its own train type, we can treat it as the absense of a train type. The way we do this is to change the function which does the 1-hot encoding to do this:
- train_to_id5('train_b') = [0, 1, 0, 0, 0]
- train_to_id4('train_b') = [0, 1, 0, 0]

This means we need to change a couple of other things as well, in particular we will use a different "loss-function", which is what the neural network uses to get errors to correct for. "categorical_crossentropy" uses the output node with the highest value, and checks if this is the same as the expected target. If the network returns [0.2, 0.1, 0.4, 0.1, 0.2] and the target is [0, 0, 1, 0, 0] ('train_c'), it will treat this as 1 correct prediction. This is only useful if the data you present only ever has 1 class in them. "binary_crossentropy" assumes that all elements can be either 0 or 1. Anything above 0.50 will be trated as if it was 1, meaning it will treat the model output [0.0, 0.2, 0.6, 0.3] where the expected target is [0, 0, 0, 1] ('train_d'), it will treat this as 50% correct, since the first two entries are correct, and the last two are incorrect.

### Exercises:
- Find the one change required to make the script run correctly when you use target_to_id4 instead of target_to_id5
- Does this increase the real validation accuracy?
- Can you think of any other reason for using this alternative method? 


In [ ]:
from helpers import train_to_id4, train_to_id5, load_dataset, plot_validation_history
from scipy.signal import welch, resample
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Change this to set how many steps long you want your time-series to be
input_length = 5000


# A function to extract the values we need as input and output for the model training
# Note: You can make changes here to look at different features
def extract_features(signals, train_types):
    model_input = []
    model_target = []
    
    # Iterate over all signals and corresponding train types
    for signal, train_type in zip(signals, train_types):
                
        # Assemble the signal one data point
        # ------- You can uncomment any single line below or combine some -------- #
        # signal = welch(signal, nperseg=input_length * 2)[1][1:]  # Compute an estimate of the Power Spectrum Density
        # signal = resample(signal, input_length)  # Stretch / Compress signal to fit
        # signal = signal / np.max(np.abs(signal))  # Min-Max Normalize
        # signal = np.sign(signal) * np.log(signal + 1.0)  # Reduce the range of the signal
        input_vector = np.reshape(signal, (-1, 1))  # special case if you have only 1 time series
    
        # Convert train type to number
        target = train_to_id5(train_type)  # Old method, returns array of 5 elements
        # target = train_to_id4(train_type)  # New method, returns array of 4 elements
        
        # Add to dataset to be fed to a machine learning algorithm
        model_input.append(input_vector)
        model_target.append(target)
    
    # Convert to a more digestable format and return the data, also makes also signals equally long
    model_input = sequence.pad_sequences(model_input, input_length)
    model_target = np.array(model_target)
    return model_input, model_target


# Load the data
training_x, training_y = load_dataset(dataset='training')
validate_x, validate_y = load_dataset(dataset='validate')

# Transform the data / extract features
training_x, training_y = extract_features(training_x, training_y)
validate_x, validate_y = extract_features(validate_x, validate_y)

# Build a Convolutional Neural Network
# ------- You can change the number of filters and kernel_size here --------- #
model = Sequential()
model.add(Conv1D(filters=4, kernel_size=5, padding='valid', input_shape=training_x.shape[1:]))
model.add(MaxPool1D(2))
model.add(Conv1D(filters=4, kernel_size=5, padding='valid'))
model.add(MaxPool1D(2))
model.add(Flatten())
model.add(Dense(units=5, activation='sigmoid'))
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Fit a model to the data. Note less epochs are needed here
# ------- You can change the number of epochs and batch_size here ----------- #
logger = model.fit(training_x, training_y, epochs=25, batch_size=16, validation_data=[validate_x, validate_y])

# Some more code needs to be used to determine the true validation accuracy when using this type of model output
correct = 0
true_positive, false_positive, true_negative, false_negative = 0, 0, 0, 0
predicted_y = model.predict(validate_x)
for i in range(len(predicted_y)):
    if np.max(predicted_y[i]) >= 0.50:  # This is a threshold to determine if any train types were identified
        if np.argmax(predicted_y[i]) == np.argmax(validate_y[i]):  # Compares model output with model target
            correct += 1
            true_positive += 1
        else:
            false_positive += 1
    else:
        if np.sum(validate_y[i]) == 0:  # Verifies that the model target is also an unknown train type
            correct += 1
            true_negative += 1
        else:
            false_negative += 1
accuracy = float(correct) / float(len(predicted_y))
plot_validation_history(logger, accuracy)
